## Using Python to Integrate MongoDB, JSON file, and MySQL into an ETL Process
In this project, I will be using the AdventureWorks toy database to simulate the necessary processes for this assignment. The database was built using the script provided int he GitHub, and the Date dimension was created in a similar way. Overall, I want to take this AdventureWorks database and split it into three or so dimension tables, generate a fact table that models the business practices, and use this fact table to model some business question that would support the businesses understanding of its own data.

Accomplish this by creating several dimension tables: time/date, customers, product, employees, inventory management
Use the dimension tables to help construct a fact table for overall business processes

In [318]:
import os
import json
import numpy
import datetime
import certifi
import pandas as pd

import pymongo
import sqlalchemy
from sqlalchemy import create_engine, text

In [319]:
print(f"Running SQL Alchemy Version: {sqlalchemy.__version__}")
print(f"Running PyMongo Version: {pymongo.__version__}")

Running SQL Alchemy Version: 2.0.29
Running PyMongo Version: 4.10.1


#### Declare & Assign Connection Variables for the MongoDB Server, the MySQL Server & Databases with which You'll be Working 

In [320]:
mysql_args = {
    "uid" : "root",
    "pwd" : "password",
    "hostname" : "localhost",
    "dbname" : "adventureworks_dw2"
}

# The 'cluster_location' must either be "atlas" or "local".
mongodb_args = {
    "user_name" : "Student",
    "password" : "OeviWNzKwpylwqmz",
    "cluster_name" : "TestEnvironment",
    "cluster_subnet" : "1zoql",
    "cluster_location" : "atlas", # "local"
    "db_name" : "adventureworks_data"
}

#### Define Functions for Getting Data From and Setting Data Into Databases

In [321]:
def get_sql_dataframe(sql_query, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    dframe = pd.read_sql(text(sql_query), connection);
    connection.close()
    
    return dframe

def create_database(**args):
    '''Create database programatically'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()    

    connection.execute(text(f"DROP DATABASE IF EXISTS {args['dbname']}"))
    connection.execute(text(f"CREATE DATABASE {args['dbname']}"))

    connection.close()
    

def set_dataframe(df, table_name, pk_column, db_operation, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()


def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe


def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
        
    mongo_client.close()

#### Populate MongoDB with Source Data
You only need to run this cell once; however, the operation is *idempotent*.  In other words, it can be run multiple times without changing the end result.

push all these json files into mongodb

In [ ]:
client = get_mongo_client(**mongodb_args)
# print(client.list_database_names())

# Gets the path of the Current Working Directory for this Notebook,
# and then Appends the 'data' directory.
data_dir = os.path.join(os.getcwd(), 'customer_data')

json_files = {"address" : 'address.json',
              "addresstype" : 'addresstype.json',
              "countryregion" : 'countryregion.json',
              "customer" : 'customer.json',
              "customeraddress" : 'customeraddress.json',
              "salesterritory" : 'salesterritory.json',
              "stateprovince" : 'stateprovince.json'
             }

In [ ]:
set_mongo_collections(client, mongodb_args["db_name"], data_dir, json_files)         

### 1.0. Create and Populate the New Dimension Tables
#### 1.1. Extract Data from the Source MongoDB Collections Into DataFrames

extract mongodb files, and load into pandas dfs

In [ ]:
client = get_mongo_client(**mongodb_args)
client.list_database_names()

['adventureworks_data',
 'northwind_purchasing',
 'sample_mflix',
 'admin',
 'local']

In [ ]:
client = get_mongo_client(**mongodb_args)
query = {} # Select all elements (columns), and all documents (rows).
collection = "address"

df_address = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_address.head(2)

,AddressID,AddressLine1,City,StateProvinceID,PostalCode
0,1,1970 Napa Ct.,Bothell,79,98011
1,2,9833 Mt. Dias Blv.,Bothell,79,98011


In [ ]:
client = get_mongo_client(**mongodb_args)
query = {} # Select all elements (columns), and all documents (rows).
collection = "addresstype"

df_addresstype = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_addresstype.head(2)

,AddressTypeID,Name
0,1,Billing
1,2,Home


In [ ]:
client = get_mongo_client(**mongodb_args)
query = {} # Select all elements (columns), and all documents (rows).
collection = "countryregion"

df_countryregion = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_countryregion.head(2)

,CountryRegionCode,Name
0,AD,Andorra
1,AE,United Arab Emirates


In [ ]:
client = get_mongo_client(**mongodb_args)
query = {} # Select all elements (columns), and all documents (rows).
collection = "customer"

df_customer = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_customer.head(2)

,CustomerID,TerritoryID,AccountNumber,CustomerType
0,1,1,AW00000001,S
1,2,1,AW00000002,S


In [ ]:
client = get_mongo_client(**mongodb_args)
query = {} # Select all elements (columns), and all documents (rows).
collection = "customeraddress"

df_customeraddress = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_customeraddress.head(2)

,CustomerID,AddressID,AddressTypeID
0,1,832,3
1,2,297,5


In [ ]:
client = get_mongo_client(**mongodb_args)
query = {} # Select all elements (columns), and all documents (rows).
collection = "salesterritory"

df_salesterritory = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_salesterritory.head(2)

,TerritoryID,Name,CountryRegionCode,Group,SalesYTD,SalesLastYear
0,1,Northwest,US,North America,5.767342e+06,3.298694e+06
1,2,Northeast,US,North America,3.857164e+06,3.607149e+06


In [ ]:
client = get_mongo_client(**mongodb_args)
query = {} # Select all elements (columns), and all documents (rows).
collection = "stateprovince"

df_stateprovince = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_stateprovince.head(2)

,StateProvinceID,StateProvinceCode,CountryRegionCode,IsOnlyStateProvinceFlag,Name,TerritoryID
0,1,AB,CA,0,Alberta,6
1,2,AK,US,0,Alaska,1


#### 1.2 Combining the dimension tables into a customer fact table

merge mongodb dfs into one larger table 

In [325]:
df_fact_customer = df_customer

In [326]:
df_dim_creator = df_salesterritory.rename(columns={"territory_key":"TerritoryID"})
df_fact_customer = pd.merge(df_fact_customer, df_dim_creator, on='TerritoryID', how='inner')
df_fact_customer.rename(columns={"TerritoryID":"territory_key"}, inplace=True)
df_fact_customer.head(2)

,CustomerID,territory_key,AccountNumber,CustomerType,Name,CountryRegionCode,Group,SalesYTD,SalesLastYear
0,1,1,AW00000001,S,Northwest,US,North America,5.767342e+06,3.298694e+06
1,2,1,AW00000002,S,Northwest,US,North America,5.767342e+06,3.298694e+06


In [327]:
df_dim_creator = df_countryregion.rename(columns={"Name":"CountryName"})
df_fact_customer = pd.merge(df_fact_customer, df_dim_creator, on='CountryRegionCode', how='inner')
df_fact_customer.drop(columns=['CountryRegionCode'], axis=1, inplace=True)
df_fact_customer.head(2)

,CustomerID,territory_key,AccountNumber,CustomerType,Name,Group,SalesYTD,SalesLastYear,CountryName
0,1,1,AW00000001,S,Northwest,North America,5.767342e+06,3.298694e+06,United States
1,2,1,AW00000002,S,Northwest,North America,5.767342e+06,3.298694e+06,United States


In [328]:
df_fact_customer = pd.merge(df_fact_customer, df_customeraddress, on='CustomerID', how='inner')
df_fact_customer.head(2)

,CustomerID,territory_key,AccountNumber,CustomerType,Name,Group,SalesYTD,SalesLastYear,CountryName,AddressID,AddressTypeID
0,1,1,AW00000001,S,Northwest,North America,5.767342e+06,3.298694e+06,United States,832,3
1,2,1,AW00000002,S,Northwest,North America,5.767342e+06,3.298694e+06,United States,297,5


In [329]:
df_dim_creator = df_addresstype.rename(columns={"Name":"AddressType"})
df_fact_customer = pd.merge(df_fact_customer, df_dim_creator, on='AddressTypeID', how='inner')
df_fact_customer.drop(columns=['AddressTypeID'], axis=1, inplace=True)
df_fact_customer.head(2)

,CustomerID,territory_key,AccountNumber,CustomerType,Name,Group,SalesYTD,SalesLastYear,CountryName,AddressID,AddressType
0,1,1,AW00000001,S,Northwest,North America,5.767342e+06,3.298694e+06,United States,832,Main Office
1,2,1,AW00000002,S,Northwest,North America,5.767342e+06,3.298694e+06,United States,833,Main Office


In [330]:
df_dim_creator = df_address.rename(columns={"AddressLine1":"Address"})
df_fact_customer = pd.merge(df_fact_customer, df_dim_creator, on='AddressID', how='inner')
df_fact_customer.drop(columns=['AddressID'], axis=1, inplace=True)
df_fact_customer.head(2)

,CustomerID,territory_key,AccountNumber,CustomerType,Name,Group,SalesYTD,SalesLastYear,CountryName,AddressType,Address,City,StateProvinceID,PostalCode
0,1,1,AW00000001,S,Northwest,North America,5.767342e+06,3.298694e+06,United States,Main Office,2251 Elliot Avenue,Seattle,79,98104
1,2,1,AW00000002,S,Northwest,North America,5.767342e+06,3.298694e+06,United States,Main Office,3207 S Grady Way,Renton,79,98055


In [331]:
df_dim_creator = df_stateprovince.rename(columns={"Name":"StateProvince"})
df_fact_customer = pd.merge(df_fact_customer, df_dim_creator, on='StateProvinceID', how='inner')
df_fact_customer.drop(columns=['StateProvinceID','StateProvinceCode','CountryRegionCode','IsOnlyStateProvinceFlag','TerritoryID'], axis=1, inplace=True)
df_fact_customer.head(2)

,CustomerID,territory_key,AccountNumber,CustomerType,Name,Group,SalesYTD,SalesLastYear,CountryName,AddressType,Address,City,PostalCode,StateProvince
0,1,1,AW00000001,S,Northwest,North America,5.767342e+06,3.298694e+06,United States,Main Office,2251 Elliot Avenue,Seattle,98104,Washington
1,2,1,AW00000002,S,Northwest,North America,5.767342e+06,3.298694e+06,United States,Main Office,3207 S Grady Way,Renton,98055,Washington


#### 1.3 Combining .csv files into one dimension table

Pulling .csv files into dataframes

In [ ]:
df_contact = pd.read_csv('salesperson_data/contact.csv', delimiter=';', quotechar='"', encoding='ISO-8859-1')
df_contact.head(2)

,ContactID,FirstName,LastName,EmailAddress,Phone
0,1,Gustavo,Achong,gustavo0@adventure-works.com,398-555-0132
1,2,Catherine,Abel,catherine0@adventure-works.com,747-555-0171


In [ ]:
df_employee = pd.read_csv('salesperson_data/employee.csv', delimiter=';', quotechar='"')
df_employee.head(2)

,EmployeeID,NationalIDNumber,ContactID,ManagerID,Title,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,CurrentFlag
0,1,14417807,1209,16,Production Technician - WC60,1972-05-15 00:00:00,M,M,1996-07-31 00:00:00,0,1
1,2,253022876,1030,6,Marketing Assistant,1977-06-03 00:00:00,S,M,1997-02-26 00:00:00,0,1


In [ ]:
df_salesperson = pd.read_csv('salesperson_data/salesperson.csv', delimiter=';', quotechar='"')
df_salesperson.head(2)

,SalesPersonID,TerritoryID,SalesQuota,Bonus,CommissionPct,SalesYTD,SalesLastYear,ModifiedDate
0,268,0,0.0,0.0,0.000,6.775585e+05,0.000000e+00,2001-01-28 00:00:00
1,275,2,300000.0,4100.0,0.012,4.557045e+06,1.750406e+06,2001-06-24 00:00:00


Merging the .csv files into one larger table

In [ ]:
df_fact_sales_employees = df_salesperson

In [ ]:
df_dim_creator = df_employee.rename(columns={"EmployeeID":"SalesPersonID"})
df_fact_sales_employees = pd.merge(df_fact_sales_employees, df_dim_creator, on='SalesPersonID', how='inner')
df_fact_sales_employees.head(2)

,SalesPersonID,TerritoryID,SalesQuota,Bonus,CommissionPct,SalesYTD,SalesLastYear,ModifiedDate,NationalIDNumber,ContactID,ManagerID,Title,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,CurrentFlag
0,268,0,0.0,0.0,0.000,6.775585e+05,0.000000e+00,2001-01-28 00:00:00,502097814,1011,273,North American Sales Manager,1941-11-17 00:00:00,M,M,2001-02-04 00:00:00,1,1
1,275,2,300000.0,4100.0,0.012,4.557045e+06,1.750406e+06,2001-06-24 00:00:00,841560125,1026,268,Sales Representative,1959-01-26 00:00:00,S,M,2001-07-01 00:00:00,1,1


In [ ]:
# df_fact_sales_employees.drop(0, axis=0).reset_index(drop=True)
# df_fact_sales_employees.head(2)

In [ ]:
df_fact_sales_employees = pd.merge(df_fact_sales_employees, df_contact, on='ContactID', how='inner')
df_fact_sales_employees.rename(columns={"TerritoryID":"territory_key"}, inplace=True)
df_fact_sales_employees.head(2)

,SalesPersonID,territory_key,SalesQuota,Bonus,CommissionPct,SalesYTD,SalesLastYear,ModifiedDate,NationalIDNumber,ContactID,...,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,CurrentFlag,FirstName,LastName,EmailAddress,Phone
0,268,0,0.0,0.0,0.000,6.775585e+05,0.000000e+00,2001-01-28 00:00:00,502097814,1011,...,1941-11-17 00:00:00,M,M,2001-02-04 00:00:00,1,1,Stephen,Jiang,stephen0@adventure-works.com,238-555-0197
1,275,2,300000.0,4100.0,0.012,4.557045e+06,1.750406e+06,2001-06-24 00:00:00,841560125,1026,...,1959-01-26 00:00:00,S,M,2001-07-01 00:00:00,1,1,Michael,Blythe,michael9@adventure-works.com,257-555-0154


In [ ]:
df_fact_sales_employees.columns

Index(['SalesPersonID', 'territory_key', 'SalesQuota', 'Bonus',
       'CommissionPct', 'SalesYTD', 'SalesLastYear', 'ModifiedDate',
       'NationalIDNumber', 'ContactID', 'ManagerID', 'Title', 'BirthDate',
       'MaritalStatus', 'Gender', 'HireDate', 'SalariedFlag', 'CurrentFlag',
       'FirstName', 'LastName', 'EmailAddress', 'Phone'],
      dtype='object')

#### 1.4 Create product dimension table from the sql database

Pull data from the sql db and populate dfs

In [339]:
sql_query = "SELECT * FROM adventureworks.salesorderheader;"
df_dim_salesorders = get_sql_dataframe(sql_query, **mysql_args)
df_dim_salesorders.head(2)

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,...,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43659,1,2001-07-01,2001-07-13,2001-07-08,5,b'\x00',SO43659,PO522145787,10-4020-000676,...,16281.0,105041Vi84182,NaN,24643.9362,1971.5149,616.0984,27231.5495,None,b'!S\xb6y\xca9\x15A\x9c\xba\x8f\xe0\x90>\x12\xe6',2001-07-08
1,43660,1,2001-07-01,2001-07-13,2001-07-08,5,b'\x00',SO43660,PO18850127500,10-4020-000117,...,5618.0,115213Vi29411,NaN,1553.1035,124.2483,38.8276,1716.1794,None,"b'-\xc4\x8ds;\xd0\xa1H\x98""\xf9Zg\xeas\x89'",2001-07-08


In [ ]:
sql_query = "SELECT * FROM adventureworks.salesorderheadersalesreason;"
df_dim_salesordersheaders = get_sql_dataframe(sql_query, **mysql_args)
df_dim_salesordersheaders.head(5)

,SalesOrderID,SalesReasonID,ModifiedDate
0,43697,5,2001-07-01
1,43697,9,2001-07-01
2,43702,5,2001-07-02
3,43702,9,2001-07-02
4,43703,5,2001-07-02


In [ ]:
sql_query = "SELECT * FROM adventureworks.salesreason;"
df_dim_salesreason = get_sql_dataframe(sql_query, **mysql_args)
df_dim_salesreason.head(2)

,SalesReasonID,Name,ReasonType,ModifiedDate
0,1,Price,Other,1998-06-01
1,2,On Promotion,Promotion,1998-06-01


In [ ]:
sql_query = "SELECT * FROM adventureworks.salesorderdetail;"
df_dim_salesorderdetail = get_sql_dataframe(sql_query, **mysql_args)
df_dim_salesorderdetail.head(2)

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43659,1,4911-403C-98,1,776,1,2024.994,0.0,2024.994,"b'm\xc9\x07\xb2\xe6\xd9+@\x84p,\xc1v\xc4""\x83'",2001-07-01
1,43659,2,4911-403C-98,3,777,1,2024.994,0.0,6074.982,"b'\r`\xbbzw\x1e\xbeA\x9f\xe5\xb9\x14,\xfc\x08\...",2001-07-01


Drop unnecessary columns and begin merging into larger table

In [340]:
df_fact_sales = df_dim_salesorderdetail.drop(columns=['ModifiedDate','rowguid'])
df_fact_sales.head(10)

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal
0,43659,1,4911-403C-98,1,776,1,2024.9940,0.0,2024.9940
1,43659,2,4911-403C-98,3,777,1,2024.9940,0.0,6074.9820
2,43659,3,4911-403C-98,1,778,1,2024.9940,0.0,2024.9940
3,43659,4,4911-403C-98,1,771,1,2039.9940,0.0,2039.9940
4,43659,5,4911-403C-98,1,772,1,2039.9940,0.0,2039.9940
5,43659,6,4911-403C-98,2,773,1,2039.9940,0.0,4079.9880
6,43659,7,4911-403C-98,1,774,1,2039.9940,0.0,2039.9940
7,43659,8,4911-403C-98,3,714,1,28.8404,0.0,86.5212
8,43659,9,4911-403C-98,1,716,1,28.8404,0.0,28.8404
9,43659,10,4911-403C-98,6,709,1,5.7000,0.0,34.2000


In [341]:
df_dim_creator = df_dim_salesorders # .rename(columns={"Name":"ReasonName"})
df_dim_creator.drop(columns=['rowguid'], inplace=True)
df_fact_sales = pd.merge(df_fact_sales, df_dim_creator, on='SalesOrderID', how='left')
df_fact_sales.head(10)

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,RevisionNumber,...,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,ModifiedDate
0,43659,1,4911-403C-98,1,776,1,2024.9940,0.0,2024.9940,1,...,5,16281.0,105041Vi84182,NaN,24643.9362,1971.5149,616.0984,27231.5495,None,2001-07-08
1,43659,2,4911-403C-98,3,777,1,2024.9940,0.0,6074.9820,1,...,5,16281.0,105041Vi84182,NaN,24643.9362,1971.5149,616.0984,27231.5495,None,2001-07-08
2,43659,3,4911-403C-98,1,778,1,2024.9940,0.0,2024.9940,1,...,5,16281.0,105041Vi84182,NaN,24643.9362,1971.5149,616.0984,27231.5495,None,2001-07-08
3,43659,4,4911-403C-98,1,771,1,2039.9940,0.0,2039.9940,1,...,5,16281.0,105041Vi84182,NaN,24643.9362,1971.5149,616.0984,27231.5495,None,2001-07-08
4,43659,5,4911-403C-98,1,772,1,2039.9940,0.0,2039.9940,1,...,5,16281.0,105041Vi84182,NaN,24643.9362,1971.5149,616.0984,27231.5495,None,2001-07-08
5,43659,6,4911-403C-98,2,773,1,2039.9940,0.0,4079.9880,1,...,5,16281.0,105041Vi84182,NaN,24643.9362,1971.5149,616.0984,27231.5495,None,2001-07-08
6,43659,7,4911-403C-98,1,774,1,2039.9940,0.0,2039.9940,1,...,5,16281.0,105041Vi84182,NaN,24643.9362,1971.5149,616.0984,27231.5495,None,2001-07-08
7,43659,8,4911-403C-98,3,714,1,28.8404,0.0,86.5212,1,...,5,16281.0,105041Vi84182,NaN,24643.9362,1971.5149,616.0984,27231.5495,None,2001-07-08
8,43659,9,4911-403C-98,1,716,1,28.8404,0.0,28.8404,1,...,5,16281.0,105041Vi84182,NaN,24643.9362,1971.5149,616.0984,27231.5495,None,2001-07-08
9,43659,10,4911-403C-98,6,709,1,5.7000,0.0,34.2000,1,...,5,16281.0,105041Vi84182,NaN,24643.9362,1971.5149,616.0984,27231.5495,None,2001-07-08


Reviewing column names and renaming to keys instead of ids

In [342]:
df_fact_sales.columns

Index(['SalesOrderID', 'SalesOrderDetailID', 'CarrierTrackingNumber',
       'OrderQty', 'ProductID', 'SpecialOfferID', 'UnitPrice',
       'UnitPriceDiscount', 'LineTotal', 'RevisionNumber', 'OrderDate',
       'DueDate', 'ShipDate', 'Status', 'OnlineOrderFlag', 'SalesOrderNumber',
       'PurchaseOrderNumber', 'AccountNumber', 'CustomerID', 'ContactID',
       'SalesPersonID', 'TerritoryID', 'BillToAddressID', 'ShipToAddressID',
       'ShipMethodID', 'CreditCardID', 'CreditCardApprovalCode',
       'CurrencyRateID', 'SubTotal', 'TaxAmt', 'Freight', 'TotalDue',
       'Comment', 'ModifiedDate'],
      dtype='object')

In [343]:
df_fact_sales = df_fact_sales.rename(columns=
    {"CustomerID":"CustomerKey", "ContactID":"ContactKey", 
     "SalesPersonID":"SalesPersonKey", "TerritoryID":"TerritoryKey", 
     "BillToAddressID":"BillToAddressKey", "ShipToAddressID":"ShipToAddressKey", 
     "ShipMethodID":"ShipMethodKey", "CreditCardID":"CreditCardKey","ProductID":"ProductKey"})
df_fact_sales.columns

Index(['SalesOrderID', 'SalesOrderDetailID', 'CarrierTrackingNumber',
       'OrderQty', 'ProductKey', 'SpecialOfferID', 'UnitPrice',
       'UnitPriceDiscount', 'LineTotal', 'RevisionNumber', 'OrderDate',
       'DueDate', 'ShipDate', 'Status', 'OnlineOrderFlag', 'SalesOrderNumber',
       'PurchaseOrderNumber', 'AccountNumber', 'CustomerKey', 'ContactKey',
       'SalesPersonKey', 'TerritoryKey', 'BillToAddressKey',
       'ShipToAddressKey', 'ShipMethodKey', 'CreditCardKey',
       'CreditCardApprovalCode', 'CurrencyRateID', 'SubTotal', 'TaxAmt',
       'Freight', 'TotalDue', 'Comment', 'ModifiedDate'],
      dtype='object')

#### 1.5 Perform Any Necessary Transformations to the DataFrames

Adding in key columns instead of just the provided ids to standardize the row ids and make them unique to the new tables

In [ ]:
print(df_fact_customer.columns)
df_fact_customer.head(2)

Index(['CustomerKey', 'CustomerID', 'territory_key', 'AccountNumber',
       'CustomerType', 'Name', 'Group', 'SalesYTD', 'SalesLastYear',
       'CountryName', 'AddressType', 'Address', 'City', 'PostalCode',
       'StateProvince'],
      dtype='object')


,CustomerKey,CustomerID,territory_key,AccountNumber,CustomerType,Name,Group,SalesYTD,SalesLastYear,CountryName,AddressType,Address,City,PostalCode,StateProvince
0,1,1,1,AW00000001,S,Northwest,North America,5.767342e+06,3.298694e+06,United States,Main Office,2251 Elliot Avenue,Seattle,98104,Washington
1,2,2,1,AW00000002,S,Northwest,North America,5.767342e+06,3.298694e+06,United States,Main Office,3207 S Grady Way,Renton,98055,Washington


In [332]:
df_fact_customer.insert(0, "CustomerKey", range(1, df_fact_customer.shape[0]+1))

df_fact_customer.head(2)

,CustomerKey,CustomerID,territory_key,AccountNumber,CustomerType,Name,Group,SalesYTD,SalesLastYear,CountryName,AddressType,Address,City,PostalCode,StateProvince
0,1,1,1,AW00000001,S,Northwest,North America,5.767342e+06,3.298694e+06,United States,Main Office,2251 Elliot Avenue,Seattle,98104,Washington
1,2,2,1,AW00000002,S,Northwest,North America,5.767342e+06,3.298694e+06,United States,Main Office,3207 S Grady Way,Renton,98055,Washington


In [344]:
print(df_fact_sales.columns)
df_fact_sales.head(2)

Index(['SalesOrderID', 'SalesOrderDetailID', 'CarrierTrackingNumber',
       'OrderQty', 'ProductKey', 'SpecialOfferID', 'UnitPrice',
       'UnitPriceDiscount', 'LineTotal', 'RevisionNumber', 'OrderDate',
       'DueDate', 'ShipDate', 'Status', 'OnlineOrderFlag', 'SalesOrderNumber',
       'PurchaseOrderNumber', 'AccountNumber', 'CustomerKey', 'ContactKey',
       'SalesPersonKey', 'TerritoryKey', 'BillToAddressKey',
       'ShipToAddressKey', 'ShipMethodKey', 'CreditCardKey',
       'CreditCardApprovalCode', 'CurrencyRateID', 'SubTotal', 'TaxAmt',
       'Freight', 'TotalDue', 'Comment', 'ModifiedDate'],
      dtype='object')


,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductKey,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,RevisionNumber,...,ShipMethodKey,CreditCardKey,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,ModifiedDate
0,43659,1,4911-403C-98,1,776,1,2024.994,0.0,2024.994,1,...,5,16281.0,105041Vi84182,NaN,24643.9362,1971.5149,616.0984,27231.5495,None,2001-07-08
1,43659,2,4911-403C-98,3,777,1,2024.994,0.0,6074.982,1,...,5,16281.0,105041Vi84182,NaN,24643.9362,1971.5149,616.0984,27231.5495,None,2001-07-08


In [345]:
df_fact_sales.insert(0, "SalesOrderKey", range(1, df_fact_sales.shape[0]+1))

df_fact_sales.head(2)

,SalesOrderKey,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductKey,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,...,ShipMethodKey,CreditCardKey,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,ModifiedDate
0,1,43659,1,4911-403C-98,1,776,1,2024.994,0.0,2024.994,...,5,16281.0,105041Vi84182,NaN,24643.9362,1971.5149,616.0984,27231.5495,None,2001-07-08
1,2,43659,2,4911-403C-98,3,777,1,2024.994,0.0,6074.982,...,5,16281.0,105041Vi84182,NaN,24643.9362,1971.5149,616.0984,27231.5495,None,2001-07-08


In [ ]:
print(df_fact_sales_employees.columns)
df_fact_sales_employees.head(2)

Index(['SalesPersonID', 'territory_key', 'SalesQuota', 'Bonus',
       'CommissionPct', 'SalesYTD', 'SalesLastYear', 'ModifiedDate',
       'NationalIDNumber', 'ContactID', 'ManagerID', 'Title', 'BirthDate',
       'MaritalStatus', 'Gender', 'HireDate', 'SalariedFlag', 'CurrentFlag',
       'FirstName', 'LastName', 'EmailAddress', 'Phone'],
      dtype='object')


,SalesPersonID,territory_key,SalesQuota,Bonus,CommissionPct,SalesYTD,SalesLastYear,ModifiedDate,NationalIDNumber,ContactID,...,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,CurrentFlag,FirstName,LastName,EmailAddress,Phone
0,268,0,0.0,0.0,0.000,6.775585e+05,0.000000e+00,2001-01-28 00:00:00,502097814,1011,...,1941-11-17 00:00:00,M,M,2001-02-04 00:00:00,1,1,Stephen,Jiang,stephen0@adventure-works.com,238-555-0197
1,275,2,300000.0,4100.0,0.012,4.557045e+06,1.750406e+06,2001-06-24 00:00:00,841560125,1026,...,1959-01-26 00:00:00,S,M,2001-07-01 00:00:00,1,1,Michael,Blythe,michael9@adventure-works.com,257-555-0154


In [ ]:
df_fact_sales_employees.insert(0, "SalesPersonKey", range(1, df_fact_sales_employees.shape[0]+1))

df_fact_sales_employees.head(2)

,SalesPersonKey,SalesPersonID,territory_key,SalesQuota,Bonus,CommissionPct,SalesYTD,SalesLastYear,ModifiedDate,NationalIDNumber,...,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,CurrentFlag,FirstName,LastName,EmailAddress,Phone
0,1,268,0,0.0,0.0,0.000,6.775585e+05,0.000000e+00,2001-01-28 00:00:00,502097814,...,1941-11-17 00:00:00,M,M,2001-02-04 00:00:00,1,1,Stephen,Jiang,stephen0@adventure-works.com,238-555-0197
1,2,275,2,300000.0,4100.0,0.012,4.557045e+06,1.750406e+06,2001-06-24 00:00:00,841560125,...,1959-01-26 00:00:00,S,M,2001-07-01 00:00:00,1,1,Michael,Blythe,michael9@adventure-works.com,257-555-0154


### 2.0 Creating and Populating Dimensions

#### 2.1 Creating dimension table

Using script from the Lab3 - 2000 -> 2010 as running larger caused SQL query disconnect

In [ ]:
sql_query = "SELECT * FROM adventureworks.dim_date;"
df_dim_date = get_sql_dataframe(sql_query, **mysql_args)
df_dim_date.head(2)

,date_key,full_date,date_name,date_name_us,date_name_eu,day_of_week,day_name_of_week,day_of_month,day_of_year,weekday_weekend,...,is_last_day_of_month,calendar_quarter,calendar_year,calendar_year_month,calendar_year_qtr,fiscal_month_of_year,fiscal_quarter,fiscal_year,fiscal_year_month,fiscal_year_qtr
0,20000101,2000-01-01,2000/01/01,01/01/2000,01/01/2000,7,Saturday,1,1,Weekend,...,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
1,20000102,2000-01-02,2000/01/02,01/02/2000,02/01/2000,1,Sunday,2,2,Weekend,...,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3


#### 2.2 Populating tables with dim date keys

In [ ]:
temp = df_fact_sales_employees

temp dfs in case something breaks for easy reset

In [ ]:
df_fact_sales_employees = temp

Replacing full dates with date keys

In [ ]:
df_dim_create_date = df_dim_date.rename(columns={"date_key" : "hire_date_key", "full_date" : "HireDate"})
df_fact_sales_employees.HireDate = df_fact_sales_employees.HireDate.astype('datetime64[ns]').dt.date
df_fact_sales_employees = pd.merge(df_fact_sales_employees, df_dim_create_date, on='HireDate', how='left')
df_fact_sales_employees.drop(['HireDate'], axis=1, inplace=True)
df_fact_sales_employees.head(2)

,SalesPersonKey,SalesPersonID,territory_key,SalesQuota,Bonus,CommissionPct,SalesYTD,SalesLastYear,ModifiedDate,NationalIDNumber,...,is_last_day_of_month,calendar_quarter,calendar_year,calendar_year_month,calendar_year_qtr,fiscal_month_of_year,fiscal_quarter,fiscal_year,fiscal_year_month,fiscal_year_qtr
0,1,268,0,0.0,0.0,0.000,6.775585e+05,0.000000e+00,2001-01-28,502097814,...,N,1,2001,2001-02,2001Q1,8,3,2001,2001-08,2001Q3
1,2,275,2,300000.0,4100.0,0.012,4.557045e+06,1.750406e+06,2001-06-24,841560125,...,N,3,2001,2001-07,2001Q3,1,1,2002,2002-01,2002Q1


In [ ]:
df_dim_create_date = df_dim_date.rename(columns={"date_key" : "modified_date_key", "full_date" : "ModifiedDate"})
df_fact_sales_employees.ModifiedDate = df_fact_sales_employees.ModifiedDate.astype('datetime64[ns]').dt.date
df_fact_sales_employees = pd.merge(df_fact_sales_employees, df_dim_create_date, on='ModifiedDate', how='left')
df_fact_sales_employees.drop(['ModifiedDate'], axis=1, inplace=True)
df_fact_sales_employees.head(2)

,SalesPersonKey,SalesPersonID,territory_key,SalesQuota,Bonus,CommissionPct,SalesYTD,SalesLastYear,NationalIDNumber,ContactID,...,is_last_day_of_month_y,calendar_quarter_y,calendar_year_y,calendar_year_month_y,calendar_year_qtr_y,fiscal_month_of_year_y,fiscal_quarter_y,fiscal_year_y,fiscal_year_month_y,fiscal_year_qtr_y
0,1,268,0,0.0,0.0,0.000,6.775585e+05,0.000000e+00,502097814,1011,...,N,1,2001,2001-01,2001Q1,7,3,2001,2001-07,2001Q3
1,2,275,2,300000.0,4100.0,0.012,4.557045e+06,1.750406e+06,841560125,1026,...,N,2,2001,2001-06,2001Q2,12,4,2001,2001-12,2001Q4


In [346]:
temp = df_fact_sales

In [ ]:
df_fact_sales = temp

In [347]:
df_dim_create_date = df_dim_date.rename(columns={"date_key" : "modified_date_key", "full_date" : "ModifiedDate"})[
    ["modified_date_key", "ModifiedDate"]
]
df_fact_sales.ModifiedDate = df_fact_sales.ModifiedDate.astype('datetime64[ns]').dt.date
df_fact_sales = pd.merge(df_fact_sales, df_dim_create_date, on='ModifiedDate', how='left')
df_fact_sales.drop(['ModifiedDate'], axis=1, inplace=True)
df_fact_sales.head(2)

,SalesOrderKey,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductKey,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,...,ShipMethodKey,CreditCardKey,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,modified_date_key
0,1,43659,1,4911-403C-98,1,776,1,2024.994,0.0,2024.994,...,5,16281.0,105041Vi84182,NaN,24643.9362,1971.5149,616.0984,27231.5495,None,20010708
1,2,43659,2,4911-403C-98,3,777,1,2024.994,0.0,6074.982,...,5,16281.0,105041Vi84182,NaN,24643.9362,1971.5149,616.0984,27231.5495,None,20010708


In [348]:
df_dim_create_date = df_dim_date.rename(columns={"date_key" : "order_date_key", "full_date" : "OrderDate"})[
    ["order_date_key", "OrderDate"]
]
df_fact_sales.OrderDate = df_fact_sales.OrderDate.astype('datetime64[ns]').dt.date
df_fact_sales = pd.merge(df_fact_sales, df_dim_create_date, on='OrderDate', how='left')
df_fact_sales.drop(['OrderDate'], axis=1, inplace=True)
df_fact_sales.head(2)

,SalesOrderKey,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductKey,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,...,CreditCardKey,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,modified_date_key,order_date_key
0,1,43659,1,4911-403C-98,1,776,1,2024.994,0.0,2024.994,...,16281.0,105041Vi84182,NaN,24643.9362,1971.5149,616.0984,27231.5495,None,20010708,20010701
1,2,43659,2,4911-403C-98,3,777,1,2024.994,0.0,6074.982,...,16281.0,105041Vi84182,NaN,24643.9362,1971.5149,616.0984,27231.5495,None,20010708,20010701


In [349]:
df_dim_create_date = df_dim_date.rename(columns={"date_key" : "due_date_key", "full_date" : "DueDate"})[
    ["due_date_key", "DueDate"]
]
df_fact_sales.DueDate = df_fact_sales.DueDate.astype('datetime64[ns]').dt.date
df_fact_sales = pd.merge(df_fact_sales, df_dim_create_date, on='DueDate', how='left')
df_fact_sales.drop(['DueDate'], axis=1, inplace=True)
df_fact_sales.head(2)

,SalesOrderKey,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductKey,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,...,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,modified_date_key,order_date_key,due_date_key
0,1,43659,1,4911-403C-98,1,776,1,2024.994,0.0,2024.994,...,105041Vi84182,NaN,24643.9362,1971.5149,616.0984,27231.5495,None,20010708,20010701,20010713
1,2,43659,2,4911-403C-98,3,777,1,2024.994,0.0,6074.982,...,105041Vi84182,NaN,24643.9362,1971.5149,616.0984,27231.5495,None,20010708,20010701,20010713


In [350]:
df_dim_create_date = df_dim_date.rename(columns={"date_key" : "ship_date_key", "full_date" : "ShipDate"})[
    ["ship_date_key", "ShipDate"]
]
df_fact_sales.ShipDate = df_fact_sales.ShipDate.astype('datetime64[ns]').dt.date
df_fact_sales = pd.merge(df_fact_sales, df_dim_create_date, on='ShipDate', how='left')
df_fact_sales.drop(['ShipDate'], axis=1, inplace=True)
df_fact_sales.head(2)

,SalesOrderKey,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductKey,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,...,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,modified_date_key,order_date_key,due_date_key,ship_date_key
0,1,43659,1,4911-403C-98,1,776,1,2024.994,0.0,2024.994,...,NaN,24643.9362,1971.5149,616.0984,27231.5495,None,20010708,20010701,20010713,20010708
1,2,43659,2,4911-403C-98,3,777,1,2024.994,0.0,6074.982,...,NaN,24643.9362,1971.5149,616.0984,27231.5495,None,20010708,20010701,20010713,20010708


### 3.0 Load DataFrames into new Data Warehouse

Loading all dataframes back into new sql warehouse

In [ ]:
create_database(**mysql_args)

In [354]:
dataframe = df_fact_sales
table_name = 'dim_sales'
primary_key = 'SalesOrderKey'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

In [ ]:
dataframe = df_fact_customer
table_name = 'dim_customer'
primary_key = 'CustomerKey'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

In [370]:
dataframe = df_fact_sales_employees
table_name = 'dim_employees'
primary_key = 'SalesPersonKey'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

In [ ]:
dataframe = df_dim_date
table_name = 'dim_date'
primary_key = 'date_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

#### 3.1 Validate that the New Tables were Created.

Pulling tables back out to ensure they are in

In [355]:
sql_query = "SELECT * FROM adventureworks_dw2.dim_sales;"
df_check = get_sql_dataframe(sql_query, **mysql_args)
df_check.head(2)

,SalesOrderKey,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductKey,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,...,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,modified_date_key,order_date_key,due_date_key,ship_date_key
0,1,43659,1,4911-403C-98,1,776,1,2024.994,0.0,2024.994,...,NaN,24643.9362,1971.5149,616.0984,27231.5495,None,20010708,20010701,20010713,20010708
1,2,43659,2,4911-403C-98,3,777,1,2024.994,0.0,6074.982,...,NaN,24643.9362,1971.5149,616.0984,27231.5495,None,20010708,20010701,20010713,20010708


In [356]:
sql_query = "SELECT * FROM adventureworks_dw2.dim_customer;"
df_check = get_sql_dataframe(sql_query, **mysql_args)
df_check.head(2)

,CustomerKey,CustomerID,territory_key,AccountNumber,CustomerType,Name,Group,SalesYTD,SalesLastYear,CountryName,AddressType,Address,City,PostalCode,StateProvince
0,1,1,1,AW00000001,S,Northwest,North America,5.767342e+06,3.298694e+06,United States,Main Office,2251 Elliot Avenue,Seattle,98104,Washington
1,2,2,1,AW00000002,S,Northwest,North America,5.767342e+06,3.298694e+06,United States,Main Office,3207 S Grady Way,Renton,98055,Washington


In [357]:
sql_query = "SELECT * FROM adventureworks_dw2.dim_employees;"
df_check = get_sql_dataframe(sql_query, **mysql_args)
df_check.head(2)

,SalesPersonKey,SalesPersonID,territory_key,SalesQuota,Bonus,CommissionPct,SalesYTD,SalesLastYear,NationalIDNumber,ContactID,...,is_last_day_of_month_y,calendar_quarter_y,calendar_year_y,calendar_year_month_y,calendar_year_qtr_y,fiscal_month_of_year_y,fiscal_quarter_y,fiscal_year_y,fiscal_year_month_y,fiscal_year_qtr_y
0,1,268,0,0.0,0.0,0.000,6.775585e+05,0.000000e+00,502097814,1011,...,N,1,2001,2001-01,2001Q1,7,3,2001,2001-07,2001Q3
1,2,275,2,300000.0,4100.0,0.012,4.557045e+06,1.750406e+06,841560125,1026,...,N,2,2001,2001-06,2001Q2,12,4,2001,2001-12,2001Q4


In [358]:
sql_query = "SELECT * FROM adventureworks_dw2.dim_date;"
df_check = get_sql_dataframe(sql_query, **mysql_args)
df_check.head(2)

,date_key,full_date,date_name,date_name_us,date_name_eu,day_of_week,day_name_of_week,day_of_month,day_of_year,weekday_weekend,...,is_last_day_of_month,calendar_quarter,calendar_year,calendar_year_month,calendar_year_qtr,fiscal_month_of_year,fiscal_quarter,fiscal_year,fiscal_year_month,fiscal_year_qtr
0,20000101,2000-01-01,2000/01/01,01/01/2000,01/01/2000,7,Saturday,1,1,Weekend,...,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
1,20000102,2000-01-02,2000/01/02,01/02/2000,02/01/2000,1,Sunday,2,2,Weekend,...,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3


### 4.0 Create overarching fact table

From these new combined tables, creating larger table that connects everything with key business elements, if you want more info expand that table

In [359]:
sql_query = "SELECT CustomerKey, CustomerType, SalesYTD FROM adventureworks_dw2.dim_customer;"
df_dim_customer_keys = get_sql_dataframe(sql_query, **mysql_args)
df_dim_customer_keys.head(2)

,CustomerKey,CustomerType,SalesYTD
0,1,S,5.767342e+06
1,2,S,5.767342e+06


In [361]:
sql_query = "SELECT SalesOrderKey, SalesOrderDetailID, ProductKey, SalesPersonKey, CustomerKey, TotalDue, OrderQty, UnitPrice, TerritoryKey FROM adventureworks_dw2.dim_sales;"
df_dim_sales_keys = get_sql_dataframe(sql_query, **mysql_args)
df_dim_sales_keys.head(2)

,SalesOrderKey,SalesOrderDetailID,ProductKey,SalesPersonKey,CustomerKey,TotalDue,OrderQty,UnitPrice,TerritoryKey
0,1,1,776,279.0,676,27231.5495,1,2024.994,5
1,2,2,777,279.0,676,27231.5495,3,2024.994,5


In [373]:
sql_query = "SELECT SalesPersonID, SalesQuota, Bonus, CommissionPct FROM adventureworks_dw2.dim_employees;"
df_dim_employees_keys = get_sql_dataframe(sql_query, **mysql_args)
df_dim_employees_keys.head(2)

,SalesPersonID,SalesQuota,Bonus,CommissionPct
0,268,0.0,0.0,0.000
1,275,300000.0,4100.0,0.012


In [374]:
df_fact = df_dim_sales_keys

In [375]:
df_fact = pd.merge(df_fact, df_dim_customer_keys, on='CustomerKey', how='inner')
df_fact.head(2)

,SalesOrderKey,SalesOrderDetailID,ProductKey,SalesPersonKey,CustomerKey,TotalDue,OrderQty,UnitPrice,TerritoryKey,CustomerType,SalesYTD
0,1,1,776,279.0,676,27231.5495,1,2024.994,5,I,5.767342e+06
1,2,2,777,279.0,676,27231.5495,3,2024.994,5,I,5.767342e+06


In [376]:
df_dim_employees_keys.head(2)

,SalesPersonID,SalesQuota,Bonus,CommissionPct
0,268,0.0,0.0,0.000
1,275,300000.0,4100.0,0.012


In [379]:
df_dim_employees_keys.rename(columns={"SalesPersonID":"SalesPersonKey"}, inplace=True)
df_fact = pd.merge(df_fact, df_dim_employees_keys, on='SalesPersonKey', how='inner')
df_fact.head(5)

,SalesOrderKey,SalesOrderDetailID,ProductKey,SalesPersonKey,CustomerKey,TotalDue,OrderQty,UnitPrice,TerritoryKey,CustomerType,SalesYTD,SalesQuota,Bonus,CommissionPct
0,1,1,776,279.0,676,27231.5495,1,2024.994,5,I,5.767342e+06,300000.0,6700.0,0.01
1,2,2,777,279.0,676,27231.5495,3,2024.994,5,I,5.767342e+06,300000.0,6700.0,0.01
2,3,3,778,279.0,676,27231.5495,1,2024.994,5,I,5.767342e+06,300000.0,6700.0,0.01
3,4,4,771,279.0,676,27231.5495,1,2039.994,5,I,5.767342e+06,300000.0,6700.0,0.01
4,5,5,772,279.0,676,27231.5495,1,2039.994,5,I,5.767342e+06,300000.0,6700.0,0.01


In [380]:
df_fact.insert(0, 'fact_id', range(1, len(df_fact) + 1))
df_fact.head(2)

,fact_id,SalesOrderKey,SalesOrderDetailID,ProductKey,SalesPersonKey,CustomerKey,TotalDue,OrderQty,UnitPrice,TerritoryKey,CustomerType,SalesYTD,SalesQuota,Bonus,CommissionPct
0,1,1,1,776,279.0,676,27231.5495,1,2024.994,5,I,5.767342e+06,300000.0,6700.0,0.01
1,2,2,2,777,279.0,676,27231.5495,3,2024.994,5,I,5.767342e+06,300000.0,6700.0,0.01


push this new dataframe back into the db and check if it got sent correctly

In [381]:
dataframe = df_fact
table_name = 'fact_table'
primary_key = 'fact_id'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

In [382]:
sql_query = "SELECT * FROM adventureworks_dw2.fact_table;"
df_check = get_sql_dataframe(sql_query, **mysql_args)
df_check.head(2)

,fact_id,SalesOrderKey,SalesOrderDetailID,ProductKey,SalesPersonKey,CustomerKey,TotalDue,OrderQty,UnitPrice,TerritoryKey,CustomerType,SalesYTD,SalesQuota,Bonus,CommissionPct
0,1,1,1,776,279.0,676,27231.5495,1,2024.994,5,I,5.767342e+06,300000.0,6700.0,0.01
1,2,2,2,777,279.0,676,27231.5495,3,2024.994,5,I,5.767342e+06,300000.0,6700.0,0.01


### 5.0 Demonstrate that the New Data Warehouse Exists and Contains the Correct Data


#### 5.1 Query
This query allows us to select territories, keys, and salespersons where we sum the total amount sold and average the order quantity, essentially allowing insights into sale volumes and performance across locations for comparisons. For example, from this data we can see that salesperson with key 3 conducts much of the sales for the company currently, indicating areas the company can look to expand through additional sales reps or increase marketing. We can also see that there are two sales people in Seattle, which makes sense given the size of the city, but we can also compare the total sales to avgorderquantity to identify if the reps are selling large volumes at lower increments, or lower volumes at higher increments.


In [ ]:
sql_query = """
SELECT 
    de.territory_key, 
    dc.city, 
    de.SalesPersonKey, 
    SUM(ft.TotalDue) AS TotalSales, 
    AVG(ft.OrderQty) AS AvgOrderQuantity
FROM 
    adventureworks_dw2.fact_table AS ft
JOIN 
    adventureworks_dw2.dim_employees AS de ON ft.SalesPersonKey = de.SalesPersonID
JOIN 
    adventureworks_dw2.dim_customer AS dc ON ft.CustomerKey = dc.CustomerKey
GROUP BY 
    de.territory_key, dc.city, de.SalesPersonKey
ORDER BY 
    TotalSales DESC
LIMIT 10;
"""

In [386]:
df_salesperson_info = get_sql_dataframe(sql_query, **mysql_args)
df_salesperson_info

,territory_key,city,SalesPersonKey,TotalSales,AvgOrderQuantity
0,3,Seattle,4,6.142563e+07,3.1743
1,10,Yakima,12,6.005733e+07,4.7481
2,4,Bellingham,3,5.988292e+07,5.1221
3,4,Redmond,3,5.929531e+07,4.0845
4,2,Seattle,2,5.652312e+07,3.6074
5,4,Everett,3,5.377781e+07,3.4771
6,10,Bellingham,12,4.960739e+07,4.2603
7,4,Bremerton,3,4.927626e+07,3.8500
8,7,Everett,13,4.635158e+07,5.2568
9,4,Puyallup,3,4.590538e+07,4.5445


#### 5.2 Query
This query allows us to view the over time sales on a monthly basis for each sales person including how much they sold and the quantity. This date-based analytics allows us to perform time sensitive review of the amounts each salesperson is selling, and understand what months and seasons are good to sell in what region, as well as what to not sell when and where.


In [387]:
sql_query = """
SELECT 
    d.calendar_year AS Year,
    d.month_name AS Month,
    ft.TerritoryKey AS Territory,
    ft.SalesPersonKey AS SalesRep,
    COUNT(ft.SalesOrderKey) AS TotalOrders,
    SUM(ft.TotalDue) AS TotalSales,
    AVG(ft.UnitPrice) AS AvgUnitPrice
FROM 
    adventureworks_dw2.fact_table AS ft
JOIN 
    adventureworks_dw2.dim_sales AS de ON ft.SalesOrderKey = de.SalesOrderKey
JOIN 
    adventureworks_dw2.dim_date AS d ON de.order_date_key = d.date_key
GROUP BY 
    d.calendar_year, d.month_name, ft.TerritoryKey, ft.SalesPersonKey
ORDER BY 
    Year ASC, Month ASC, Territory ASC, TotalSales ASC;
"""

In [388]:
df_sales_overtime = get_sql_dataframe(sql_query, **mysql_args)
df_sales_overtime

,Year,Month,Territory,SalesRep,TotalOrders,TotalSales,AvgUnitPrice
0,2001,August,1,281.0,5,5.403821e+04,1627.135200
1,2001,August,1,268.0,11,2.993133e+05,1120.745845
2,2001,August,1,276.0,10,6.539676e+05,1515.672750
3,2001,August,1,283.0,28,8.236553e+05,997.429632
4,2001,August,1,280.0,80,2.588774e+06,814.929429
...,...,...,...,...,...,...,...
684,2004,May,8,289.0,177,1.063748e+07,316.424454
685,2004,May,9,288.0,22,3.696230e+05,392.225182
686,2004,May,9,290.0,63,2.510011e+06,601.221524
687,2004,May,10,284.0,1,8.914618e+02,672.294000
